In [1]:
from metadrive.envs.metadrive_env import MetaDriveEnv
from metadrive.policy.expert_policy import ExpertPolicy
from metadrive.examples.ppo_expert.torch_expert import torch_expert as expert

from IPython.display import Image, clear_output
from pprint import pprint

In [2]:
def run_scenario_n_times_correct(n_scenarios=10, seed=0, expert_driving=True) -> list:
    """
    Runs same scenario n time and collects the traces
    """

    traces = []
    try:

        for rep in range(n_scenarios):
            env = MetaDriveEnv(config={"map": "C", "num_scenarios": n_scenarios})

            obs, step_info = env.reset(seed)
            step_info["repetition"] = rep
            traces.append(step_info)
            print(f"{env.current_seed = }")
            while True:

                # get action from expert driving, or a dummy action
                action = (
                    expert(env.agent, deterministic=True)
                    if expert_driving
                    else [0, 0.33]
                )
                obs, reward, tm, tr, step_info = env.step(action)
                step_info["repetition"] = rep
                traces.append(step_info)

                if tm or tr:
                    break

            env.close()
    finally:
        pass

    return traces

In [3]:
import pandas as pd


def are_traces_deterministic(traces) -> bool:
    df = pd.DataFrame(traces)

    # grouping by repetition to get a list of traces
    traces = df.groupby("repetition")

    # drop index and repetition ID to compare only step info later
    stripped_traces = [
        trace.reset_index(drop=True).drop("repetition", axis=1) for _, trace in traces
    ]

    # iterate over each trace and check if it is equal to the first one
    are_equal_to_first_trace = [
        trace.equals(stripped_traces[0]) for trace in stripped_traces
    ]

    # if all traces are equal to the first, then all traces are equal hence deterministic
    return all(are_equal_to_first_trace)

In [4]:
seed = 7

deterministic_traces = run_scenario_n_times_correct(seed=seed, expert_driving=True)
clear_output()

In [5]:
are_traces_deterministic(deterministic_traces)

True

In [17]:
env = MetaDriveEnv(config={"map": "C", "num_scenarios": 1})
_, _ = env.reset(0)

[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 1


In [30]:
obs, reward, tm, tr, step_info = env.step([0.1, 0.121])
step_info

{'overtake_vehicle_num': 0,
 'velocity': 0.36716062897956786,
 'steering': 0.1,
 'acceleration': 0.121,
 'step_energy': 0.0011212138640949176,
 'episode_energy': 0.006269344789094084,
 'policy': 'EnvInputPolicy',
 'action': [0.1, 0.121],
 'raw_action': (0.1, 0.121),
 'crash_vehicle': False,
 'crash_object': False,
 'crash_building': False,
 'crash_human': False,
 'crash_sidewalk': False,
 'out_of_road': False,
 'arrive_dest': False,
 'max_step': False,
 'env_seed': 0,
 'crash': False,
 'step_reward': 0.03566454048787876,
 'cost': 0,
 'episode_reward': 0.200821612444963,
 'episode_length': 11}

In [19]:
def get_npc_velocity(env):
    npcs = env.agent_manager.get_objects()
    npcs = list(npcs.values())
    npc = npcs[-1]
    return npc.velocity[0]

In [33]:
get_npc_velocity(env)

0.009618991985917091

In [10]:
npcs = env.agent_manager.get_objects()
npcs = list(npcs.values())

In [11]:
env.close()

In [12]:
traces = []
try:

    for rep in range(10):
        env = MetaDriveEnv(config={"map": "C", "num_scenarios": 10})

        obs, step_info = env.reset(5)
        step_info["repetition"] = rep
        step_info["vel"] = get_npc_velocity(env)
        traces.append(step_info)
        print(f"{env.current_seed = }")
        env.agent_manager.get_state()
        while True:

            # get action from expert driving, or a dummy action
            action = expert(env.agent, deterministic=True) if True else [0, 0.33]
            obs, reward, tm, tr, step_info = env.step(action)
            step_info["repetition"] = rep
            step_info["vel"] = get_npc_velocity(env)
            traces.append(step_info)

            if tm or tr:
                break

        env.close()
finally:
    pass

[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 10


env.current_seed = 5


[INFO] Episode ended! Scenario Index: 5 Reason: arrive_dest.
[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 10


env.current_seed = 5


[INFO] Episode ended! Scenario Index: 5 Reason: arrive_dest.
[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 10


...

[INFO] Episode ended! Scenario Index: 5 Reason: arrive_dest.
[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 10


env.current_seed = 5


[INFO] Episode ended! Scenario Index: 5 Reason: arrive_dest.
[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 10


env.current_seed = 5


[INFO] Episode ended! Scenario Index: 5 Reason: arrive_dest.


In [16]:
are_traces_deterministic(traces)

True